# IA financière + Prompt Engineering (sortie texte)

Ce parcours couvre :
1) Rédiger un prompt efficace pour l’analyse financière  
2) Tester le prompt sur un **tableau** (KPIs calculés)  
3) Créer un premier script Python avec l’API OpenAI (réponse texte)  
4) Transformer un prompt en **outil réutilisable**  
5) Créer une **interface Gradio** pour tester plusieurs prompts



## 1) Rédiger un prompt efficace pour l’analyse financière

**Objectif** : construire des prompts **clairs** et **contextualisés** pour analyser des **données financières**.  
**Idées clés** :
- **Rôle & contexte** : précisez le rôle de l’IA (ex. *analyste financier senior*) et le contexte (secteur, période, type de données).  
- **Tâche** : décrivez exactement ce que vous voulez (résumé, KPIs, risques, recommandations).  
- **Contraintes** : ton professionnel, concision, citer les chiffres avec unité/devise, éviter le jargon inutile.  
- **Format de sortie** : imposez **JSON** (clé/valeurs) pour exploiter facilement la réponse dans du code.


In [ ]:
PROMPT_TEMPLATE = """Vous etes un(e) analyste financier(ere) senior.
Contexte : {context}
Donnees fournies : {data_desc}

Taches (sortie texte) :
1) Resume (5 lignes max) clair et actionnable.
2) KPIs clefs (3 puces) au format : Nom - Valeur Unite/Devise (citer le chiffre exact).
3) Risques (2 puces) avec justification d'une ligne.
4) Actions proposees (3 puces) pour un decideur non specialiste.

Contraintes :
- Ton professionnel et concis ; eviter le jargon.
- Toujours citer les chiffres avec unite/devise (ex. M USD, %, etc.).
- Sortie en TEXTE : titres + listes a puces, pas de JSON.
"""
PROMPT_TEMPLATE


# Préparation et importations

In [ ]:
!pip install yfinance openai pandas numpy matplotlib gradio

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
# ---------------------- Import Yahoo Finance (Adj Close) ----------------------
UNIVERSE   = ["AAPL","MSFT","JPM","XOM"]   # modifiez librement
START_DATE = "2020-01-01"
END_DATE   = None  # jusqu'a aujourd'hui

# Telechargement des donnees (on suit exactement votre snippet)
data = yf.download(UNIVERSE, start=START_DATE, end=END_DATE, progress=False, auto_adjust=False)

# Si plusieurs tickers, on prend la colonne 'Adj Close'
if isinstance(data.columns, pd.MultiIndex):
    prices = data["Adj Close"].copy()           # colonnes = tickers
else:
    prices = data[["Adj Close"]].copy()         # colonnes = ["Adj Close"]
    if isinstance(UNIVERSE, list) and len(UNIVERSE) == 1:
        prices.columns = [UNIVERSE[0]]          # renommer la colonne unique en nom du ticker

# Apercu brut
print("Apercu du tableau de prix (Adj Close) :")
print(prices.head())
# ------------------------------------------------------------------------------

# KPIs sur la base des prix
rets = np.log(prices).diff()
win = 20  # fenetre pour volatilite realisee (approx. 1 mois de bourse)

kpis = []
for col in prices.columns:
    s = prices[col].dropna()
    r = rets[col].dropna()
    last_price = float(s.iloc[-1]) if len(s) else np.nan
    r_1m = float(np.exp(r.iloc[-21:].sum()) - 1) if len(r) >= 21 else np.nan
    r_3m = float(np.exp(r.iloc[-63:].sum()) - 1) if len(r) >= 63 else np.nan
    vol20 = float(r.iloc[-win:].std() * np.sqrt(252)) if len(r) >= win else np.nan
    kpis.append({
        "ticker": col,
        "last_price": last_price,
        "return_1m": r_1m,
        "return_3m": r_3m,
        "vol20_annualized": vol20
    })

df_kpis = pd.DataFrame(kpis)
display(df_kpis)

# Formatter propre pour injection dans le prompt
k = df_kpis.copy()
k["last_price"]        = k["last_price"].map(lambda x: f"{x:,.2f} USD" if pd.notnull(x) else "NA")
k["return_1m"]         = k["return_1m"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")
k["return_3m"]         = k["return_3m"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")
k["vol20_annualized"]  = k["vol20_annualized"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")

DATA_DESC_YF = "KPIs (Yahoo Finance):\n" + k.to_string(index=False)
print("\n=== Description pour le prompt ===\n", DATA_DESC_YF)


In [ ]:
# Imports communs
import os, json, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (8,4)
plt.rcParams['axes.grid'] = True

# Détection  de la clé API
api_key = os.getenv("OPENAI_API_KEY", "")
print("OPENAI_API_KEY défini :", api_key[:4])

In [ ]:

from openai import OpenAI

client = OpenAI(api_key=api_key)

In [ ]:
CONTEXT = "Portefeuille multi-secteurs, horizon court terme."
prompt_yf = PROMPT_TEMPLATE.format(context=CONTEXT, data_desc=DATA_DESC_YF)

resp_yf = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role":"user","content": prompt_yf}],
    temperature=0.2,
)
print(resp_yf.choices[0].message.content)


In [ ]:
def run_prompt(context: str, data_desc: str, extra: str = "", model: str = "gpt-4o-mini", temperature: float = 0.2) -> str:
    """Construit le prompt et renvoie la reponse TEXTE (pas de JSON)."""
    prompt_here = PROMPT_TEMPLATE.format(context=context, data_desc=data_desc)
    if extra:
        prompt_here += "\n\nConsignes additionnelles :\n" + extra
    r = client.chat.completions.create(
        model=model,
        messages=[{"role":"user","content": prompt_here}],
        temperature=float(temperature),
    )
    return r.choices[0].message.content

# Exemple rapide : rejouer sur les KPIs Yahoo
print(run_prompt(CONTEXT, DATA_DESC_YF, extra="Souligne les divergences sectorielles et propose 3 actions."))


In [ ]:
import gradio as gr

def ui_from_yahoo(tickers_csv, start, end, context, extra, model, temperature):
    # Parse tickers
    UNIVERSE = [t.strip().upper() for t in tickers_csv.split(",") if t.strip()]
    START_DATE = start or "2020-01-01"
    END_DATE   = end if end not in ("", None) else None

    # Import EXACTEMENT comme votre snippet (Adj Close)
    data = yf.download(UNIVERSE, start=START_DATE, end=END_DATE, progress=False, auto_adjust=False)
    if isinstance(data.columns, pd.MultiIndex):
        prices = data["Adj Close"].copy()
    else:
        prices = data[["Adj Close"]].copy()
        if len(UNIVERSE) == 1:
            prices.columns = [UNIVERSE[0]]

    prices = prices.dropna(how="all")
    rets = np.log(prices).diff()

    # KPIs
    win = 20
    kpis = []
    for col in prices.columns:
        s = prices[col].dropna()
        r = rets[col].dropna()
        last_price = float(s.iloc[-1]) if len(s) else np.nan
        r_1m = float(np.exp(r.iloc[-21:].sum()) - 1) if len(r) >= 21 else np.nan
        r_3m = float(np.exp(r.iloc[-63:].sum()) - 1) if len(r) >= 63 else np.nan
        vol20 = float(r.iloc[-win:].std() * np.sqrt(252)) if len(r) >= win else np.nan
        kpis.append({"ticker": col, "last_price": last_price, "return_1m": r_1m, "return_3m": r_3m, "vol20_annualized": vol20})

    df_k = pd.DataFrame(kpis).copy()
    df_k["last_price"]       = df_k["last_price"].map(lambda x: f"{x:,.2f} USD" if pd.notnull(x) else "NA")
    df_k["return_1m"]        = df_k["return_1m"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")
    df_k["return_3m"]        = df_k["return_3m"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")
    df_k["vol20_annualized"] = df_k["vol20_annualized"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")

    data_desc = "KPIs (Yahoo Finance):\n" + df_k.to_string(index=False)
    txt = run_prompt(context, data_desc, extra=extra, model=model, temperature=temperature)
    return txt, data_desc

with gr.Blocks() as demo:
    gr.Markdown("# Yahoo Finance -> KPIs -> Prompt (sortie texte)")
    with gr.Row():
        tickers_in = gr.Textbox(label="Tickers (CSV)", value="AAPL,MSFT,JPM,XOM")
        start_in   = gr.Textbox(label="Start (YYYY-MM-DD)", value="2020-01-01")
        end_in     = gr.Textbox(label="End (YYYY-MM-DD ou vide)", value="")
    context_in = gr.Textbox(label="Contexte", value="Portefeuille multi-secteurs, horizon court terme.", lines=2)
    extra_in   = gr.Textbox(label="Consignes additionnelles", value="Souligne les divergences sectorielles et propose 3 actions.", lines=2)
    with gr.Row():
        model_in = gr.Dropdown(label="Modele", choices=["gpt-4o-mini","gpt-4o"], value="gpt-4o-mini")
        temp_in  = gr.Slider(label="Temperature", minimum=0.0, maximum=1.0, step=0.1, value=0.2)

    run_btn = gr.Button("Analyser")
    out_text = gr.Textbox(label="Reponse (texte)", lines=14)
    out_prompt = gr.Code(label="Tableau KPIs injecte", language="markdown")

    run_btn.click(ui_from_yahoo,
                  inputs=[tickers_in, start_in, end_in, context_in, extra_in, model_in, temp_in],
                  outputs=[out_text, out_prompt])

# Pour lancer l'interface :
demo.launch()


## Défi — Refaire l’application **100% local** avec Ollama

**Objectif.** Reprendre votre mini-application (KPIs Yahoo Finance → prompt engineering → réponse texte) et la refaire **sans aucun appel cloud**, en utilisant **uniquement Ollama** en local.

### Contraintes
- **Modèle** : un LLM open-source lancé via Ollama (ex. `llama3:8b`, `mistral:7b-instruct`, `qwen2.5:7b-instruct`).
- **Package** : utiliser le package Python `ollama` (`pip install ollama`), pas l’API OpenAI.
- **Données** : récupérer les prix avec **Yahoo Finance** (`yfinance`), calculer au minimum : **dernier cours**, **rendement 1 mois**, **rendement 3 mois**, **volatilité 20j annualisée**.
- **Prompt** : réutiliser le **template en sortie texte** (titres + puces, **pas de JSON**), en injectant votre tableau de KPIs formaté.
- **Sortie** : texte clair en français (résumé ≤ 5 lignes, 3 KPIs en puces, 2 risques en puces, 3 actions en puces).

### À faire (checklist)
1. **Installer** et démarrer Ollama, puis tirer un modèle :  
   `ollama pull llama3:8b`
2. **Calculer les KPIs** (Yahoo Finance) et construire un texte `data_desc` lisible (tableau formaté).
3. **Construire le prompt** (même structure que précédemment : contexte + `data_desc` + consignes).
4. **Générer localement** avec `ollama.chat` (sortie **texte** uniquement, pas de JSON).
5. **Comparer** rapidement la réponse locale vs la version cloud : clarté, précision chiffrée, ton.

### Livrables attendus
- Le **script/notebook** complet (import YF, KPIs, prompt, appel `ollama.chat`).
- Le **prompt final** injecté (copié en clair).
- La **réponse texte** du modèle (résumé, 3 KPIs, 2 risques, 3 actions).
- 3–5 lignes de **retour d’expérience** (vitesse, qualité, limites, pistes d’amélioration : quantisation, autre modèle, séparation par secteurs…).

> Astuce : si le modèle « oublie » les chiffres, **raccourcissez** le tableau (top 4–6 tickers), **réitérez** la consigne « citer les chiffres avec unité/devise » et baissez la **température** (≈ 0.2).


# LLM local
ollama pull llama3:8b
# Librairies Python
pip install ollama yfinance pandas numpy matplotlib


In [ ]:
%pip install ollama yfinance pandas numpy matplotlib

In [ ]:
!ollama list

In [ ]:
# ============================================================
# Mini-application IA financière — Solution 100% locale (Ollama)
# KPIs Yahoo Finance -> Prompt engineering (texte) -> Réponse locale
# AUCUN appel cloud (OpenAI non utilisé)
# ============================================================

import re
import numpy as np
import pandas as pd
import yfinance as yf
import ollama

# ------------------ Paramètres ------------------
TICKERS     = ["AAPL", "MSFT", "JPM", "XOM"]   # modifiez votre univers
START_DATE  = "2020-01-01"
END_DATE    = None
INTERVAL    = "1d"                             # "1d","1wk","1mo"
LLM_MODEL   = "llama3.1:8b"                      # ex. "llama3:8b", "mistral:7b-instruct", "qwen2.5:7b-instruct"
TEMP        = 0.2

# ------------------ 1) Import YF (Adj Close) ------------------
raw = yf.download(
    TICKERS,
    start=START_DATE,
    end=END_DATE,
    interval=INTERVAL,
    auto_adjust=False,     # on lit bien 'Adj Close'
    progress=False,
    group_by="column"
)

if isinstance(raw.columns, pd.MultiIndex):
    prices = raw["Adj Close"].copy()           # colonnes = tickers
else:
    prices = raw[["Adj Close"]].copy()         # colonnes = ["Adj Close"]
    if isinstance(TICKERS, list) and len(TICKERS) == 1:
        prices.columns = [TICKERS[0]]

prices = prices.dropna(how="all")
print("Aperçu des prix (Adj Close) :")
print(prices.head())

# ------------------ 2) KPIs sur les prix ------------------
# Rendements log
rets = np.log(prices).diff()

# KPIs : dernier cours, performance 1m (~21 j) / 3m (~63 j), vol 20j annualisée
win = 20
rows = []
for col in prices.columns:
    s = prices[col].dropna()
    r = rets[col].dropna()
    last_price = float(s.iloc[-1]) if len(s) else np.nan
    r_1m = float(np.exp(r.iloc[-21:].sum()) - 1) if len(r) >= 21 else np.nan
    r_3m = float(np.exp(r.iloc[-63:].sum()) - 1) if len(r) >= 63 else np.nan
    vol20 = float(r.iloc[-win:].std() * np.sqrt(252)) if len(r) >= win else np.nan
    rows.append({
        "ticker": col,
        "last_price": last_price,
        "return_1m": r_1m,
        "return_3m": r_3m,
        "vol20_annualized": vol20
    })

df_kpis = pd.DataFrame(rows)
print("\nKPIs bruts :")
print(df_kpis)

# Mise en forme pour un tableau lisible à injecter dans le prompt
k = df_kpis.copy()
k["last_price"]        = k["last_price"].map(lambda x: f"{x:,.2f} USD" if pd.notnull(x) else "NA")
k["return_1m"]         = k["return_1m"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")
k["return_3m"]         = k["return_3m"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")
k["vol20_annualized"]  = k["vol20_annualized"].map(lambda x: f"{x*100:.2f} %" if pd.notnull(x) else "NA")

DATA_DESC = "KPIs (Yahoo Finance):\n" + k.to_string(index=False)
print("\n=== Données injectées dans le prompt ===\n", DATA_DESC)

# ------------------ 3) Prompt template (sortie TEXTE) ------------------
PROMPT_TEMPLATE = """Vous etes un(e) analyste financier(ere) senior.
Contexte : {context}
Donnees fournies : {data_desc}

Taches (sortie texte) :
1) Resume (5 lignes max) clair et actionnable.
2) KPIs clefs (3 puces) au format : Nom - Valeur Unite/Devise (citer le chiffre exact).
3) Risques (2 puces) avec justification d'une ligne.
4) Actions proposees (3 puces) pour un decideur non specialiste.

Contraintes :
- Ton professionnel et concis ; eviter le jargon.
- Toujours citer les chiffres avec unite/devise (ex. M USD, %, etc.).
- Sortie en TEXTE : titres + listes a puces + commentaires et interprétations
"""

CONTEXT = "Portefeuille multi-secteurs, horizon court terme."
prompt = PROMPT_TEMPLATE.format(context=CONTEXT, data_desc=DATA_DESC)

# ------------------ 4) Génération locale avec Ollama ------------------
system_msg = (
    "Tu es un analyste financier. Reponds uniquement a partir des donnees fournies. "
    "Sortie en TEXTE (titres + puces + commentaires et interprétations) "
    "Cite clairement les chiffres avec leur unite/devise."
)

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user",   "content": prompt}
]

print("\n=== Prompt (récapitulatif) ===\n")
print(prompt[:1000] + ("\n... (tronqué)" if len(prompt) > 1000 else ""))

resp = ollama.chat(
    model=LLM_MODEL,
    messages=messages,
    options={"temperature": TEMP}
)

print("\n=== Réponse du modèle (Ollama local) ===\n")
print(resp["message"]["content"])

# ============================================================
# Fin — Solution locale : aucun appel cloud, tout en local.
# ============================================================
